In [1]:
using FITSIO
using RecipesBase
using Statistics
using LinearAlgebra

# =============================================================================
# Mission Support and Calibration
# =============================================================================

"""
Dictionary of simple conversion functions for different missions
"""
const SIMPLE_CALIBRATION_FUNCS = Dict{String, Function}(
    "nustar" => (pi) -> pi * 0.04 + 1.62,
    "xmm" => (pi) -> pi * 0.001,
    "nicer" => (pi) -> pi * 0.01,
    "ixpe" => (pi) -> pi / 375 * 15,
    "axaf" => (pi) -> (pi - 1) * 14.6e-3
)

"""
Abstract type for mission-specific calibration and interpretation
"""
abstract type AbstractMissionSupport end

"""
Structure containing mission-specific calibration and interpretation information
"""
struct MissionSupport{T} <: AbstractMissionSupport
    name::String
    instrument::Union{String, Nothing}
    epoch::Union{T, Nothing}
    calibration_func::Function
    interpretation_func::Union{Function, Nothing}
end

function get_mission_support(mission::String, 
                           instrument::Union{String, Nothing}=nothing,
                           epoch::Union{Float64, Nothing}=nothing)
    mission_lower = lowercase(mission)
    
    calib_func = if haskey(SIMPLE_CALIBRATION_FUNCS, mission_lower)
        SIMPLE_CALIBRATION_FUNCS[mission_lower]
    else
        throw(ArgumentError("Mission $mission not recognized"))
    end
    
    MissionSupport{Float64}(mission_lower, instrument, epoch, calib_func, nothing)
end

function apply_calibration(mission_support::MissionSupport, pi_channels::AbstractArray{T}) where T
    mission_support.calibration_func.(pi_channels)
end

function patch_mission_info(info::Dict{String,Any}, mission::Union{String,Nothing}=nothing)
    if isnothing(mission)
        return info
    end
    
    mission_lower = lowercase(mission)
    patched_info = copy(info)
    
    if mission_lower == "xmm" && haskey(patched_info, "gti")
        patched_info["gti"] = string(patched_info["gti"], ",GTI0")
    elseif mission_lower == "xte" && haskey(patched_info, "ecol")
        patched_info["ecol"] = "PHA"
        patched_info["ccol"] = "PCUID"
    end
    
    return patched_info
end

function interpret_fits_data!(f::FITS, mission_support::MissionSupport)
    # Placeholder for mission-specific interpretation
    # This would contain mission-specific FITS handling logic
    return nothing
end

# =============================================================================
# Data Structures
# =============================================================================

"""
Abstract type for all event list implementations
"""
abstract type AbstractEventList{T} end

"""
    DictMetadata

A structure containing metadata from FITS file headers.

## Fields

- `headers::Vector{Dict{String,Any}}`: A vector of dictionaries containing header information from each HDU.
"""
struct DictMetadata
    headers::Vector{Dict{String,Any}}
end

"""
    EventList{T} <: AbstractEventList{T}

A structure containing event data from a FITS file.

## Fields

- `filename::String`: Path to the source FITS file.
- `times::Vector{T}`: Vector of event times.
- `energies::Union{Vector{T}, Nothing}`: Vector of event energies (or nothing if not available).
- `extra_columns::Dict{String, Vector}`: Dictionary of additional column data.
- `metadata::DictMetadata`: Metadata information extracted from the FITS file headers.
"""
struct EventList{T} <: AbstractEventList{T}
    filename::String
    times::Vector{T}
    energies::Union{Vector{T}, Nothing}
    extra_columns::Dict{String, Vector}
    metadata::DictMetadata
end

# Simplified constructors
function EventList{T}(filename, times, metadata) where T
    EventList{T}(filename, times, nothing, Dict{String, Vector}(), metadata)
end

function EventList{T}(filename, times, energies, metadata) where T
    EventList{T}(filename, times, energies, Dict{String, Vector}(), metadata)
end

# Basic accessor functions
times(ev::EventList) = ev.times
energies(ev::EventList) = ev.energies

# =============================================================================
# Optimized Event Reading Function
# =============================================================================

"""
    readevents(path::String; kwargs...)

Optimized event reader that focuses on TIME as essential column and efficient memory usage.
Only processes the HDU containing TIME column to avoid unnecessary iterations.

# Arguments
- `path::String`: Path to the FITS file
- `mission`: Name of the mission (e.g., "nustar", "xmm", "nicer")
- `instrument`: Specific instrument onboard the mission
- `epoch`: Observation epoch in MJD
- `T`: Numeric type for the data (default: Float64)
- `energy_alternatives`: Column names to try for energy data
- `sector_column`: Column name for sector/detector ID data
- `skip_bitarrays`: Whether to skip BitArray columns

# Returns
- `EventList{T}` containing the extracted data

# Notes
- Only processes the HDU containing TIME column
- Optimized for memory usage and performance
- Supports mission-specific calibration
"""
function readevents(path::String;
                   mission::Union{String,Nothing}=nothing,
                   instrument::Union{String,Nothing}=nothing,
                   epoch::Union{Float64,Nothing}=nothing,
                   T::Type=Float64,
                   energy_alternatives::Vector{String}=["ENERGY", "PI", "PHA"],
                   sector_column::Union{String,Nothing}=nothing,
                   skip_bitarrays::Bool=true)
    
    # Initialize containers
    headers = Dict{String,Any}[]
    times = T[]
    energies = T[]
    extra_columns = Dict{String, Vector}()
    
    # Get mission support if specified
    mission_support = if !isnothing(mission)
        get_mission_support(mission, instrument, epoch)
    else
        nothing
    end
    
    # Track if we've found the event HDU
    event_hdu_found = false
    
    FITS(path, "r") do f
        # Apply mission-specific interpretation if available
        if !isnothing(mission_support)
            interpret_fits_data!(f, mission_support)
        end
        
        for i = 1:length(f)
            hdu = f[i]
            
            # Always collect headers
            header_dict = Dict{String,Any}()
            for key in keys(read_header(hdu))
                header_dict[string(key)] = read_header(hdu)[key]
            end
            
            # Apply mission-specific patches to header information
            if !isnothing(mission)
                header_dict = patch_mission_info(header_dict, mission)
            end
            push!(headers, header_dict)
            
            # Only process TableHDUs and stop if we've already found the event HDU
            if !event_hdu_found && isa(hdu, TableHDU)
                colnames = FITSIO.colnames(hdu)
                
                # Check for TIME column
                if "TIME" in colnames
                    # Found event HDU - process all relevant columns at once
                    event_hdu_found = true
                    @info "Found event HDU with TIME column in HDU $i"
                    
                    # Pre-allocate column matrix for better memory efficiency
                    num_rows = size(read(hdu, "TIME"))[1]
                    @info "Processing $num_rows events"
                    
                    # Read TIME column first (essential)
                    times = convert(Vector{T}, read(hdu, "TIME"))
                    
                    # Determine which energy column to use
                    energy_col = nothing
                    for ecol in energy_alternatives
                        if ecol in colnames
                            energy_col = ecol
                            @info "Using $ecol column for energy data"
                            break
                        end
                    end
                    
                    # Read energy data with mission-specific calibration if available
                    if !isnothing(energy_col)
                        raw_energy = read(hdu, energy_col)
                        energies = if !isnothing(mission_support)
                            @info "Applying mission calibration for $(mission_support.name)"
                            convert(Vector{T}, apply_calibration(mission_support, raw_energy))
                        else
                            convert(Vector{T}, raw_energy)
                        end
                    end
                    
                    # Read sector/detector information if specified
                    if !isnothing(sector_column) && sector_column in colnames
                        try
                            extra_columns["SECTOR"] = read(hdu, sector_column)
                            @info "Read sector/detector data from $sector_column"
                        catch e
                            @warn "Failed to read sector column: $e"
                        end
                    end
                    
                    # Break after processing event HDU
                    break
                end
            end
        end
    end
    
    # Check if we found any events
    if isempty(times)
        @warn "No TIME column found in FITS file $(basename(path))"
    else
        @info "Successfully loaded $(length(times)) events from $(basename(path))"
        if !isempty(energies)
            energy_range = (minimum(energies), maximum(energies))
            @info "Energy data available: $(length(energies)) values, range: $energy_range"
        end
    end
    
    # Create metadata
    metadata = DictMetadata(headers)
    
    # Create and return EventList
    return EventList{T}(path, 
                       times, 
                       isempty(energies) ? nothing : energies,
                       extra_columns, 
                       metadata)
end

# =============================================================================
# EventList Interface Methods
# =============================================================================

Base.length(ev::AbstractEventList) = length(times(ev))
Base.size(ev::AbstractEventList) = (length(ev),)

function Base.getindex(ev::EventList, i)
    if isnothing(ev.energies)
        return (ev.times[i], nothing)
    else
        return (ev.times[i], ev.energies[i])
    end
end

function Base.show(io::IO, ev::EventList{T}) where T
    energy_status = isnothing(ev.energies) ? "no energy data" : "with energy data"
    extra_cols = length(keys(ev.extra_columns))
    print(io, "EventList{$T}(n=$(length(ev)), $energy_status, $extra_cols extra columns, file=$(basename(ev.filename)))")
end

"""
    validate(events::AbstractEventList)

Validate the event list structure.
"""
function validate(events::AbstractEventList)
    evt_times = times(events)
    if !issorted(evt_times)
        throw(ArgumentError("Event times must be sorted in ascending order"))
    end
    if length(evt_times) == 0
        throw(ArgumentError("Event list is empty"))
    end
    return true
end

"""
    get_column(events::EventList, column_name::String)

Get a specific column from the event list.
"""
function get_column(events::EventList, column_name::String)
    if column_name == "TIME"
        return events.times
    elseif column_name == "ENERGY" && !isnothing(events.energies)
        return events.energies
    elseif haskey(events.extra_columns, column_name)
        return events.extra_columns[column_name]
    else
        return nothing
    end
end

"""
    list_columns(events::EventList)

List all available columns in the event list.
"""
function list_columns(events::EventList)
    columns = ["TIME"]
    if !isnothing(events.energies)
        push!(columns, "ENERGY")
    end
    append!(columns, collect(keys(events.extra_columns)))
    return sort!(unique(columns))
end

"""
    get_metadata(events::EventList, hdu_index::Int=1)

Get metadata from a specific HDU.
"""
function get_metadata(events::EventList, hdu_index::Int=1)
    if hdu_index <= length(events.metadata.headers)
        return events.metadata.headers[hdu_index]
    else
        @warn "HDU index $hdu_index not available. File has $(length(events.metadata.headers)) HDUs."
        return Dict{String,Any}()
    end
end

# =============================================================================
# Optimized Plotting Recipe
# =============================================================================

# """
#     f(el::EventList{T}, bin_size::Real=0.1; kwargs...) where T

# optimized recipe for plotting EventList as light curve using pre-allocated 2D matrices.
# Designed to handle millions of events efficiently with minimal memory allocation and garbage collection.

# # Arguments
# - `el`: EventList containing time and energy data
# - `bin_size`: Size of time bins for the light curve
# - `tstart`: Optional starting time
# - `tseg`: Optional time segment length
# - `show_errors`: Whether to display Poisson error bars
# - `show_gaps`: Whether to highlight data gaps
# - `gap_threshold`: Multiplier of median time difference for gap detection
# - `axis_limits`: Optional array of [xmin, xmax, ymin, ymax]

# # Returns
# - Optimized 2D Matrix containing plot data
# """
# @recipe function f(el::EventList{T}, bin_size::Real=0.1;
#                   tstart=nothing, tseg=nothing,
#                   show_errors=true, show_gaps=false,
#                   gap_threshold=10.0, axis_limits=nothing) where T
    
#     if isempty(el.times)
#         @warn "Event list contains no time data. Cannot create light curve."
#         return Matrix{T}(undef, 2, 0)
#     end
    
#     # Convert inputs to correct type
#     bin_size_t = convert(T, bin_size)
#     start_time = isnothing(tstart) ? minimum(el.times) : convert(T, tstart)
#     end_time = isnothing(tseg) ? maximum(el.times) : start_time + convert(T, tseg)
    
#     # Get total number of events for performance logging
#     n_events = length(el.times)
#     if n_events > 1_000_000
#         @info "Processing $(n_events) events - optimized for large datasets"
#     end
    
#     # Efficient filtering using binary search for large datasets
#     start_idx = searchsortedfirst(el.times, start_time)
#     end_idx = searchsortedlast(el.times, end_time)
    
#     if start_idx > end_idx
#         @warn "No events in specified time range [$start_time, $end_time]"
#         return Matrix{T}(undef, 2, 0)
#     end
    
#     # Use views to avoid copying large arrays
#     filtered_times = @view el.times[start_idx:end_idx]
#     n_filtered = length(filtered_times)
    
#     # Calculate optimal number of bins
#     time_span = end_time - start_time
#     n_bins = max(1, ceil(Int, time_span / bin_size_t))
    
#     if n_bins > 100_000
#         @info "Creating light curve with $n_bins bins"
#     end
    
#     # Pre-allocate arrays with exact size needed
#     bin_centers = Vector{T}(undef, n_bins)
#     counts = zeros(Int, n_bins)
    
#     # Compute bin centers efficiently using broadcasting
#     @inbounds for i in 1:n_bins
#         bin_centers[i] = start_time + (i - 0.5) * bin_size_t
#     end
    
#     # Ultra-fast vectorized binning
#     if n_filtered > 0
#         # Calculate bin indices for all events at once
#         bin_indices = @. floor(Int, (filtered_times - start_time) / bin_size_t) + 1
        
#         # Count events in each bin using efficient loop
#         @inbounds for bin_idx in bin_indices
#             if 1 <= bin_idx <= n_bins
#                 counts[bin_idx] += 1
#             end
#         end
#     end
    
#     # Set plot attributes
#     title --> "Light Curve"
#     xlabel --> "Time"
#     ylabel --> "Counts"
    
#     # Handle axis limits
#     if !isnothing(axis_limits) && length(axis_limits) == 4
#         xmin, xmax, ymin, ymax = axis_limits
#         xlims --> (isnothing(xmin) ? :auto : xmin, isnothing(xmax) ? :auto : xmax)
#         ylims --> (isnothing(ymin) ? :auto : ymin, isnothing(ymax) ? :auto : ymax)
#     end
    
#     # Optimized gap detection for very large datasets
#     if show_gaps && n_filtered > 1
#         @info "Detecting data gaps..."
        
#         # Use statistical sampling for very large datasets to speed up gap detection
#         sample_size = min(50_000, n_filtered)  # Increased sample size for better accuracy
#         if n_filtered > sample_size
#             step = max(1, n_filtered ÷ sample_size)
#             sample_indices = 1:step:n_filtered
#             sample_times = filtered_times[sample_indices]
#         else
#             sample_times = filtered_times
#         end
        
#         # Fast gap detection using efficient algorithms
#         if length(sample_times) > 1
#             time_diffs = diff(sample_times)
#             if !isempty(time_diffs)
#                 # Use robust statistics for threshold calculation
#                 median_diff = median(time_diffs)
#                 threshold = gap_threshold * median_diff
                
#                 # Find significant gaps efficiently
#                 gap_mask = time_diffs .> threshold
#                 gap_indices = findall(gap_mask)
#                 n_gaps = length(gap_indices)
                
#                 if n_gaps > 0
#                     @info "Found $n_gaps significant data gaps"
#                     max_count = maximum(counts)
                    
#                     # Pre-allocate result matrix for light curve + gaps
#                     # Light curve: 2*n_bins points (steppost), Gaps: 5*n_gaps points (rectangles)
#                     total_points = 2 * n_bins + 5 * n_gaps
#                     result = Matrix{T}(undef, 2, total_points)
                    
#                     # Fill light curve data (steppost format) - vectorized where possible
#                     @inbounds for i in 1:n_bins
#                         col1 = 2*(i-1) + 1
#                         col2 = col1 + 1
                        
#                         x_left = start_time + (i-1) * bin_size_t
#                         x_right = x_left + bin_size_t
                        
#                         result[1, col1] = x_left
#                         result[1, col2] = x_right
#                         result[2, col1] = counts[i]
#                         result[2, col2] = counts[i]
#                     end
                    
#                     # Fill gap rectangles efficiently
#                     gap_start_col = 2 * n_bins + 1
                    
#                     @inbounds for (gap_idx, time_idx) in enumerate(gap_indices)
#                         if gap_idx <= n_gaps && time_idx < length(sample_times)
#                             base_col = gap_start_col + 5*(gap_idx-1)
                            
#                             gap_start = sample_times[time_idx]
#                             gap_end = sample_times[time_idx + 1]
                            
#                             # Create rectangle: [start,end,end,start,start] x [0,0,max,max,0]
#                             result[1, base_col:base_col+4] .= [gap_start, gap_end, gap_end, gap_start, gap_start]
#                             result[2, base_col:base_col+4] .= [0, 0, max_count, max_count, 0]
#                         end
#                     end
                    
#                     # Set attributes for combined visualization
#                     seriestype --> :path
#                     linecolor --> [:blue :red]
#                     linewidth --> [1 2]
#                     label --> ["Light Curve" "Data Gaps"]
#                     fillrange --> [nothing 0]
#                     fillalpha --> [0 0.2]
#                     fillcolor --> [:auto :red]
                    
#                     return result
#                 end
#             end
#         end
#     end
    
#     # Standard light curve output paths
#     if show_errors
#         # Calculate Poisson errors efficiently
#         errors = sqrt.(max.(1, counts))  # Use max(1, counts) to avoid zero errors
        
#         # 3-row matrix: [times; counts; errors]
#         result = Matrix{T}(undef, 3, n_bins)
#         result[1, :] .= bin_centers
#         result[2, :] .= counts
#         result[3, :] .= errors
        
#         seriestype --> :scatter
#         yerror --> (@view result[3, :])
#         markersize --> 2
#         markerstrokewidth --> 0
#         label --> "Light Curve (Poisson errors, $(bin_size_t)s bins)"
        
#         return result
#     else
#         # 2-row matrix: [times; counts] - most efficient format
#         result = Matrix{T}(undef, 2, n_bins)
#         result[1, :] .= bin_centers
#         result[2, :] .= counts
        
#         seriestype --> :steppost
#         linewidth --> 1
#         color --> :blue
#         label --> "Light Curve ($(bin_size_t)s bins, $(n_filtered) events)"
        
#         return result
#     end
# end
@recipe function f(el::EventList{T}, bin_size::Real=0.1; 
                  tstart=nothing, tseg=nothing, show_errors=true, 
                  show_gaps=false, gap_threshold=10.0, axis_limits=nothing,
                  max_events_for_gaps=1_000_000) where T
    
    bin_size_t = convert(T, bin_size)
    
    if isempty(el.times)
        @warn "Event list contains no time data. Cannot create light curve."
        return [], []
    end
    
    # Use the optimized create_lightcurve function
    lc = create_lightcurve(el, bin_size_t; tstart=tstart, tseg=tseg)
    
    # Apply axis limits if specified
    if !isnothing(axis_limits) && length(axis_limits) == 4
        xmin, xmax, ymin, ymax = axis_limits
        
        if !isnothing(xmin) && !isnothing(xmax)
            xlims --> (xmin, xmax)
        elseif !isnothing(xmin)
            xlims --> (xmin, :auto)
        elseif !isnothing(xmax)
            xlims --> (:auto, xmax)
        end
        
        if !isnothing(ymin) && !isnothing(ymax)
            ylims --> (ymin, ymax)
        elseif !isnothing(ymin)
            ylims --> (ymin, :auto)
        elseif !isnothing(ymax)
            ylims --> (:auto, ymax)
        end
    end
    
    # Handle gap visualization (with safety check for large datasets)
    if show_gaps
        if length(el.times) > max_events_for_gaps
            @warn "Dataset too large for gap analysis ($(length(el.times)) events). Set max_events_for_gaps higher or use show_gaps=false for better performance."
            show_gaps = false
        else
            # Optimized gap detection
            gaps = detect_gaps_fast(el.times, gap_threshold, tstart, tseg)
            
            @series begin
                title := "Light Curve with Gaps Highlighted"
                xlabel := "Time"
                ylabel := "Counts"
                seriestype := :steppost
                linewidth := 2
                color := :blue
                label := "Light Curve (bin size: $bin_size_t)"
                lc.timebins, lc.counts
            end
            
            # Add gap visualization
            if !isempty(gaps)
                max_count = isempty(lc.counts) ? one(T) : maximum(lc.counts)
                
                for (i, (start, stop)) in enumerate(gaps)
                    @series begin
                        seriestype := :shape
                        fillalpha := 0.3
                        fillcolor := :red
                        linecolor := :red
                        label := i == 1 ? "Gaps (threshold: $(round(gap_threshold, digits=2))x median)" : nothing
                        [start, stop, stop, start, start], 
                        [zero(T), zero(T), max_count, max_count, zero(T)]
                    end
                end
            end
            
            return [], []
        end
    end
    
    # Standard light curve plot
    title --> "Light Curve"
    xlabel --> "Time"
    ylabel --> "Counts"
    
    if show_errors
        seriestype --> :scatter
        yerror --> lc.count_error
        label --> "Light Curve with Poisson errors"
    else
        seriestype --> :steppost
        label --> "Light Curve (bin size: $bin_size_t)"
    end
    
    return lc.timebins, lc.counts
end

# =============================================================================
# Light Curve Data Structures and Functions
# =============================================================================

"""
Abstract type for all light curve implementations.
"""
abstract type AbstractLightCurve{T} end

"""
    EventProperty{T}

A structure to hold additional event properties beyond time and energy.
"""
struct EventProperty{T}
    name::Symbol
    values::Vector{T}
    unit::String
end

"""
    LightCurveMetadata

A structure containing metadata for light curves.
"""
struct LightCurveMetadata
    telescope::String
    instrument::String
    object::String
    mjdref::Float64
    time_range::Tuple{Float64,Float64}
    bin_size::Float64
    headers::Vector{Dict{String,Any}}
    extra::Dict{String,Any}
end

"""
    LightCurve{T} <: AbstractLightCurve{T}

A structure representing a binned time series with additional properties.
"""
struct LightCurve{T} <: AbstractLightCurve{T}
    timebins::Vector{T}
    bin_edges::Vector{T}
    counts::Vector{Int}
    count_error::Vector{T}
    exposure::Vector{T}
    properties::Vector{EventProperty}
    metadata::LightCurveMetadata
    err_method::Symbol
end

"""
    calculate_errors(counts::Vector{Int}, method::Symbol, exposure::Vector{T}; 
                    gaussian_errors::Union{Nothing,Vector{T}}=nothing) where T

Calculate statistical uncertainties for count data.
"""
function calculate_errors(counts::Vector{Int}, method::Symbol, exposure::Vector{T}; 
                         gaussian_errors::Union{Nothing,Vector{T}}=nothing) where T
    if method === :poisson
        # For Poisson statistics: σ = sqrt(N)
        # Use sqrt(N + 1) when N = 0 to avoid zero errors
        return convert.(T, [c == 0 ? sqrt(1) : sqrt(c) for c in counts])
    elseif method === :gaussian
        if isnothing(gaussian_errors)
            throw(ArgumentError("Gaussian errors must be provided by user when using :gaussian method"))
        end
        if length(gaussian_errors) != length(counts)
            throw(ArgumentError("Length of gaussian_errors must match length of counts"))
        end
        return gaussian_errors
    else
        throw(ArgumentError("Unsupported error method: $method. Use :poisson or :gaussian"))
    end
end

"""
    create_lightcurve(eventlist::EventList{T}, binsize::Real; kwargs...) where T

Create a light curve from an event list with filtering capabilities.
"""
function create_lightcurve(
    eventlist::EventList{T}, 
    binsize::Real;
    err_method::Symbol=:poisson,
    gaussian_errors::Union{Nothing,Vector{T}}=nothing,
    tstart::Union{Nothing,Real}=nothing,
    tstop::Union{Nothing,Real}=nothing,
    tseg::Union{Nothing,Real}=nothing,  # Added for compatibility with your fast version
    filters::Dict{Symbol,Any}=Dict{Symbol,Any}(),
    fast_mode::Bool=true  # New parameter to control optimization level
) where T
    
    if isempty(eventlist.times)
        throw(ArgumentError("Event list is empty"))
    end
    
    if binsize <= 0
        throw(ArgumentError("Bin size must be positive"))
    end
    
    binsize_t = convert(T, binsize)
    
    # Determine time range efficiently
    min_time = minimum(eventlist.times)
    max_time = maximum(eventlist.times)
    
    # Handle both tstop and tseg parameters for compatibility
    start_time = isnothing(tstart) ? min_time : convert(T, tstart)
    end_time = if !isnothing(tstop)
        convert(T, tstop)
    elseif !isnothing(tseg)
        start_time + convert(T, tseg)
    else
        max_time
    end
    
    # Early validation
    if start_time >= end_time
        throw(ArgumentError("Invalid time range: start_time >= end_time"))
    end
    
    # Create bin structure efficiently (matching your fast approach)
    start_bin = floor(start_time / binsize_t) * binsize_t
    num_bins = ceil(Int, (end_time - start_bin) / binsize_t)
    
    # Pre-allocate all arrays at once
    counts = zeros(Int, num_bins)
    bin_centers = Vector{T}(undef, num_bins)
    
    # Vectorized bin center calculation
    @inbounds for i in 1:num_bins
        bin_centers[i] = start_bin + (i - 0.5) * binsize_t
    end
    
    # Fast time filtering using your optimized approach
    filtered_times, filtered_energies, valid_indices = if fast_mode
        # Use efficient filtering for large datasets
        if issorted(eventlist.times)
            start_idx = searchsortedfirst(eventlist.times, start_time)
            end_idx = searchsortedlast(eventlist.times, end_time)
            
            time_view = @view eventlist.times[start_idx:end_idx]
            energy_view = isnothing(eventlist.energies) ? nothing : @view eventlist.energies[start_idx:end_idx]
            indices_view = start_idx:end_idx
            
            (time_view, energy_view, indices_view)
        else
            # Fallback for unsorted data
            mask = (eventlist.times .>= start_time) .& (eventlist.times .<= end_time)
            indices = findall(mask)
            
            (eventlist.times[mask], 
             isnothing(eventlist.energies) ? nothing : eventlist.energies[mask],
             indices)
        end
    else
        # Standard filtering with all filters applied
        mask = (eventlist.times .>= start_time) .& (eventlist.times .<= end_time)
        
        # Apply additional filters
        for (key, value) in filters
            if key == :energy && !isnothing(eventlist.energies)
                if value isa Tuple && length(value) == 2
                    energy_mask = (eventlist.energies .>= value[1]) .& (eventlist.energies .< value[2])
                    mask .&= energy_mask
                end
            end
        end
        
        indices = findall(mask)
        (eventlist.times[mask], 
         isnothing(eventlist.energies) ? nothing : eventlist.energies[mask],
         indices)
    end
    
    # Ultra-fast vectorized binning (your approach)
    if !isempty(filtered_times)
        # Calculate all bin indices at once
        bin_indices = floor.(Int, (filtered_times .- start_bin) ./ binsize_t) .+ 1
        
        # Filter valid indices
        valid_mask = (bin_indices .>= 1) .& (bin_indices .<= num_bins)
        valid_bin_indices = bin_indices[valid_mask]
        
        # Count events efficiently
        @inbounds for idx in valid_bin_indices
            counts[idx] += 1
        end
    end
    
    # Create bin edges for full compatibility
    edges = [start_bin + i * binsize_t for i in 0:num_bins]
    
    # Calculate exposures and errors efficiently
    exposure = fill(binsize_t, num_bins)
    errors = calculate_errors(counts, err_method, exposure; gaussian_errors=gaussian_errors)
    
    # Fast property calculation
    properties = Vector{EventProperty}()
    
    # Add mean energy per bin if available and not in fast_mode
    if !fast_mode && !isempty(filtered_times) && !isnothing(filtered_energies)
        energy_bins = zeros(T, num_bins)
        energy_counts = zeros(Int, num_bins)
        
        # Vectorized energy binning
        bin_indices = floor.(Int, (filtered_times .- start_bin) ./ binsize_t) .+ 1
        valid_mask = (bin_indices .>= 1) .& (bin_indices .<= num_bins)
        
        @inbounds for (i, (bin_idx, energy)) in enumerate(zip(bin_indices, filtered_energies))
            if valid_mask[i]
                energy_bins[bin_idx] += energy
                energy_counts[bin_idx] += 1
            end
        end
        
        mean_energy = zeros(T, num_bins)
        @inbounds for i in eachindex(mean_energy)
            mean_energy[i] = energy_counts[i] > 0 ? energy_bins[i] / energy_counts[i] : zero(T)
        end
        
        push!(properties, EventProperty{T}(:mean_energy, mean_energy, "keV"))
    end
    
    # Create minimal metadata for fast mode
    extra = if fast_mode
        Dict{String,Any}(
            "filtered_nevents" => length(filtered_times),
            "total_nevents" => length(eventlist.times),
            "fast_mode" => true
        )
    else
        Dict{String,Any}(
            "filtered_nevents" => length(filtered_times),
            "total_nevents" => length(eventlist.times),
            "applied_filters" => filters,
            "fast_mode" => false
        )
    end
    
    metadata = LightCurveMetadata(
        get(eventlist.metadata.headers[1], "TELESCOP", ""),
        get(eventlist.metadata.headers[1], "INSTRUME", ""),
        get(eventlist.metadata.headers[1], "OBJECT", ""),
        get(eventlist.metadata.headers[1], "MJDREF", 0.0),
        (start_time, end_time),
        binsize_t,
        eventlist.metadata.headers,
        extra
    )
    
    return LightCurve{T}(
        bin_centers,
        collect(edges),
        counts,
        errors,
        exposure,
        properties,
        metadata,
        err_method
    )
end

"""
    create_lightcurve_simple(eventlist::EventList{T}, binsize::Real; kwargs...) where T

Ultra-fast light curve creation that returns a simple named tuple.
Matches the performance of your original fast function but with better structure.
"""
function create_lightcurve_simple(
    eventlist::EventList{T}, 
    binsize::Real;
    tstart::Union{Nothing,Real}=nothing,
    tseg::Union{Nothing,Real}=nothing,
    tstop::Union{Nothing,Real}=nothing
) where T
    
    if isempty(eventlist.times)
        throw(ArgumentError("Event list is empty"))
    end
    
    if binsize <= 0
        throw(ArgumentError("Bin size must be positive"))
    end
    
    binsize_t = convert(T, binsize)
    
    # Determine time range
    min_time = minimum(eventlist.times)
    max_time = maximum(eventlist.times)
    
    start_time = isnothing(tstart) ? min_time : convert(T, tstart)
    end_time = if !isnothing(tstop)
        convert(T, tstop)
    elseif !isnothing(tseg)
        start_time + convert(T, tseg)
    else
        max_time
    end
    
    # Create bin structure
    start_bin = floor(start_time / binsize_t) * binsize_t
    num_bins = ceil(Int, (end_time - start_bin) / binsize_t)
    
    # Pre-allocate arrays
    counts = zeros(Int, num_bins)
    bin_centers = Vector{T}(undef, num_bins)
    
    # Calculate bin centers
    @inbounds for i in 1:num_bins
        bin_centers[i] = start_bin + (i - 0.5) * binsize_t
    end
    
    # Efficient time filtering
    filtered_times = if issorted(eventlist.times)
        start_idx = searchsortedfirst(eventlist.times, start_time)
        end_idx = searchsortedlast(eventlist.times, end_time)
        @view eventlist.times[start_idx:end_idx]
    else
        filter(t -> start_time <= t <= end_time, eventlist.times)
    end
    
    # Vectorized binning
    if !isempty(filtered_times)
        bin_indices = floor.(Int, (filtered_times .- start_bin) ./ binsize_t) .+ 1
        valid_mask = (bin_indices .>= 1) .& (bin_indices .<= num_bins)
        valid_indices = bin_indices[valid_mask]
        
        @inbounds for idx in valid_indices
            counts[idx] += 1
        end
    end
    
    # Calculate errors (Poisson)
    errors = sqrt.(float.(counts))
    exposure = fill(binsize_t, num_bins)
    
    return (
        timebins = bin_centers,
        counts = counts,
        count_error = errors,
        exposure = exposure,
        bin_size = binsize_t,
        time_range = (start_time, end_time),
        nevents = length(filtered_times)
    )
end

# Basic LightCurve interface
Base.length(lc::LightCurve) = length(lc.counts)
Base.size(lc::LightCurve) = (length(lc),)
Base.getindex(lc::LightCurve, i) = (lc.timebins[i], lc.counts[i])

# =============================================================================
# Export main functions this for my purpose :)
# =============================================================================

export readevents, EventList, DictMetadata
export create_lightcurve, LightCurve, EventProperty, LightCurveMetadata
export validate, get_column, list_columns, get_metadata
export get_mission_support, apply_calibration